Context managers
================

**Outline**:
1. The notion of context
2. Custom context managers
3. Available context managers
4. The tryforce of error handling
5. Singleton pattern
6. Closing words

## 1. The notion of context

:hourglass: 20 min

Consider the following example:

In [6]:
import sys
import io

stdout_ = sys.stdout
stdout_buffer = io.StringIO()
try:
    sys.stdout = stdout_buffer

    # Code you actually want to run
    print("This is a call to the print function")
    
finally:
    sys.stdout = stdout_


In [7]:
stdout_buffer.getvalue()

'This is a call to the print function\n'

In this example, the `try`/`finally` construct defines a *context*. The goal of the context is to isolate some part of the execution. In this specific example, the isolation aims at capturing (notably) the `print` calls.

The first few statements of the function define the *setup* of the context (saving the default print buffer, overwriting the default print buffer). The whole of the `finally` block defines the *teardown* of the context (restoring the default print buffer).

Those kinds of contexts are often used in programs to make sure the teardown part will be executed properly, even when an exception occurs in the middle of the code.

Examples include:
- resource management: making sure a resource (file descriptor, lock/semaphore, etc.) are release properly;
- buffering: making sure a buffer is flushed;
- transaction: ensuring atomicity and consistency;
- monckeypatching: restoring the proper code (cf. example);
- parametrization: setting some configuration for the context (see eg. Airflow, PyTorch)

Although the notion of context is very useful, the above code has some drawbacks:
- the code of the context is mingled with the rest of the code, making it unclear which is which;
- the context is not re-usable for another function.

Python provides a very elegant mechanism to encapsulate the context code via the `with` keyword:

In [18]:
import contextlib
import io


print("Before context")
with contextlib.redirect_stdout(io.StringIO()) as buffer:
    print("This is a call to the print function")

print("After context")
print(f"Buffer: {buffer.getvalue()}")

Before context
After context
Buffer: This is a call to the print function



The `redirect_stdout` has a much more limited impact on the code, there is no more mingling of contextual and application code, and the code is now re-usable!

## 2. Custom context managers

### The short syntax

:hourglass: 10 min

Python provides two syntaxes for creating context managers. The short version makes use of the `contexlib` decorator and is quite straightforward. You decorate a single-element generator:

In [19]:
from typing import Optional, Iterator
import io
import contextlib
import sys

@contextlib.contextmanager
def redirect_stdout(buffer: Optional[io.StringIO] = None) -> Iterator[io.StringIO]:
    if buffer is None:
        buffer = io.StringIO()
    
    stdout_ = sys.stdout
    try:
        sys.stdout = buffer
        yield buffer
    finally:
        sys.stdout = stdout_


print("Before context")
with redirect_stdout() as buffer:
    print("This is a call to the print function")

print("After context")
print(f"Buffer: {buffer.getvalue()}")

Before context
After context
Buffer: This is a call to the print function



### The powerful syntax

:hourglass: 10 min


While useful for simple tasks, you sometimes want the full flexibility of having an object serving as a context manager. This can be done via the `__enter__` and `__exit__` pair of methods.

In [20]:
from typing import Optional
from types import TracebackType
import io
import sys


class STDOutCatcher:
    def __init__(self, buffer: Optional[io.StringIO] = None) -> None:
        if buffer is None:
            buffer = io.StringIO()

        self._buffer = buffer
        self._sysstdout: Optional[io.StringIO] = None

    def __enter__(self) -> io.StringIO:
        self._sysstdout = sys.stdout
        sys.stdout = self._buffer
        return self._buffer
    
    def __exit__(
        self,
        exc_type: Optional[type[BaseException]],
        exc_val: Optional[BaseException],
        exc_tb: Optional[TracebackType],
    ) -> None:
        sys.stdout = self._sysstdout


print("Before context")
with STDOutCatcher() as buffer:
    print("This is a call to the print function")

print("After context")
print(f"Buffer: {buffer.getvalue()}")

Before context
After context
Buffer: This is a call to the print function



### Exercise

:hourglass: 30 min

Create a context manager to log the start/end of a portion of code, as well as the completion time. You can use the `from time import perf_counter` to track the duration. 



:coffee: 10 min

## 3. Available context managers

:hourglass: 10 min

Some context managers are provided in the `contextlib` module:
- `closing`: call the `close` method of the object at teardown;
- `suppress`: swallow exceptions;
- `redirect_stderr`: same as `redirect_stdout` but for the error flow;

There are also a few other tools available for context managers. See https://docs.python.org/3/library/contextlib.html for more details.

## 4. The tryforce of error handling

:hourglass: 20 min

Can you predict what will be printed:

In [ ]:
def runtime_stop():
    raise RuntimeError()

try:
    print("the cat")
    runtime_stop()
    print("the dog")
except RuntimeError:
    print("the duck")
except ValueError:
    print("the lion")
except (RuntimeError, ValueError):
    print("the fox")
except Exception:
    print("the fish")
finally:
    print("the mouse")

What is the difference between the following snippets?

In [ ]:
try:
    runtime_stop()
except RuntimeError as e:
    raise e


In [ ]:
try:
    runtime_stop()
except RuntimeError as e:
    raise


In [ ]:
try:
    runtime_stop()
except RuntimeError as e:
    raise RuntimeError(2) from e

In [ ]:
import sys

try:
    runtime_stop()
except RuntimeError as e:
    tb = sys.exc_info()[2]
    raise RuntimeError(2).with_traceback(tb)

**always** provide the except clause with an exception class:

In [36]:
try:
    raise KeyboardInterrupt()
except:
    pass

In [ ]:
try:
    raise KeyboardInterrupt()
except Exception:
    pass

It is possible to create custom exceptions by inheriting from an exception class (see https://docs.python.org/3.9/library/exceptions.html#inheriting-from-built-in-exceptions).

Although this is encouraged in some languages, not so much in Python, where the exception hierarchy already provides comprehensive classes. In particular, it is worth knowing the following:
- **AssertionError**: raised when an assertion is violated (raised via the assert statement);
- **AttributeError**: raised when trying to access an attribute of an object which does not exist;
- **RuntimeError**: raised when something occurs at runtime and no more specific exception can be used;
- **NotImplementedError**: raised by an abstract method (*);
- **TypeError**: raised when the object is not of the proper type for some operation;
- **ValueError**: raised when the value is inappropriate (but the type is correct).

Some remarks:
- (*) the `NotImplementedError` exception must not be used to restrict what a class can do (cf. the substitution principle);
- Do not confuse `NotImplementedError` and `NotImplemented`;
- some exceptions serve as control flow (eg. `GeneratorExit`, `StopIteration`);
- :skull: Python 3.11 provides support for "exception groups" (https://peps.python.org/pep-0654/).

## 5. Singleton pattern

### Exposition

:hourglass: 20 min

Sometimes (rarely) it is useful to have only one instance of a class created. 

:warning: Singletons somewhat go against the nature of OOP. You should make sure that (i) it does actually make sense to create a singleton class, and (ii) this is clearly telegraphed to the user.

A few examples of sensible singletons are the following:
- logging (although in Python you have a hierarchy of singletons);
- global configuration settings;
- caching;
- limited resource (eg. sharing a thread pool).

The following example illustrates how to create an event listener Singleton:

In [22]:
from __future__ import annotations

from dataclasses import dataclass
import datetime as dt
from typing import Optional, Any, List, Iterator

@dataclass
class Event:
    message: str
    context: str
    timestamp: dt.datetime

    @classmethod
    def create(cls, message: str, context: str) -> Event:
        return cls(message, context, dt.datetime.now())
    

class EventListener:
    _instance: Optional[EventListener] = None
    _events: List[Event] = []

    def __new__(cls, *args: Any, **kwargs: Any) -> EventListener:
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, context: str) -> None:
        self._context = context

    def register(self, message: str) -> None:
        self._events.append(Event.create(message, self._context))

    def __iter__(self) -> Iterator[Event]:
        return iter(self._events)
    


el_c1 = EventListener("context 1")
el_c1.register("Entry 1")
el_c1.register("Entry 2")


el_c2 = EventListener("context 2")
el_c2.register("Entry A")
el_c2.register("Entry B")

list(EventListener(""))

[Event(message='Entry 1', context='context 1', timestamp=datetime.datetime(2024, 3, 5, 16, 5, 19, 502472)),
 Event(message='Entry 2', context='context 1', timestamp=datetime.datetime(2024, 3, 5, 16, 5, 19, 502489)),
 Event(message='Entry A', context='context 2', timestamp=datetime.datetime(2024, 3, 5, 16, 5, 19, 502507)),
 Event(message='Entry B', context='context 2', timestamp=datetime.datetime(2024, 3, 5, 16, 5, 19, 502516))]

In OOP, classes have (at least) one constructor. In Python, we usually refer to the `__init__` method as the constructor, whereas it actually serves to *initialize* (ie. fill in) the instance. The memory allocation is done by the `__new__` method.

Note that even though the `__new__` method returns an existing instance the second time, the `__init__` method is still invoked, as can be seen by the change in context. That is why we also had to make the `_events` list a class variable.

Contrast this with the following implementation of the Singleton:

In [26]:
class EventListener:
    _instance: Optional[EventListener] = None
    
    @classmethod
    def create(cls, *args: Any, **kwargs: Any) -> EventListener:
        if cls._instance is None:
            cls._instance = cls(*args, **kwargs)
        return cls._instance
    
    def __init__(self, context: str) -> None:
        self._events: List[Event] = []
        self._context = context

    def register(self, message: str) -> None:
        self._events.append(Event.create(message, self._context))

    def __iter__(self) -> Iterator[Event]:
        return iter(self._events)
    


el_c1 = EventListener.create("context 1")
el_c1.register("Entry 1")
el_c1.register("Entry 2")


el_c2 = EventListener.create("context 2")
el_c2.register("Entry A")
el_c2.register("Entry B")

list(EventListener.create(""))

[Event(message='Entry 1', context='context 1', timestamp=datetime.datetime(2024, 3, 5, 16, 15, 52, 897427)),
 Event(message='Entry 2', context='context 1', timestamp=datetime.datetime(2024, 3, 5, 16, 15, 52, 897465)),
 Event(message='Entry A', context='context 1', timestamp=datetime.datetime(2024, 3, 5, 16, 15, 52, 897512)),
 Event(message='Entry B', context='context 1', timestamp=datetime.datetime(2024, 3, 5, 16, 15, 52, 897535))]

Notice how we can use the `_events` instance variable, but the context is never updated.

Since in Python it is not possible to restrict the visibility of the constructor (like in eg. Java), the second approach might lead to inconsistency if an instance is created directly, rather than through the factory. 

> :skull: In an event-driven application, event listening is usually conducted via another pattern: the *observer* pattern. This allows for greater flexibility regarding which objects listens to what.

:coffee: 10 min

### Exercise

:hourglass: 30 min

Adapt the telemetric context manager to report to the event listener. You should have two types of events: context entrance and context exit. The exit events should hold the duration. It should be possible to generate a kind of report of the events.

> :skull: an interesting design pattern when it comes to generate reports to offer some flexibility of rendering (plain text, html, etc.) is the *visitor* pattern.

## 6. Closing words

In this module we discussed context managers, the benefits of the Python syntax (re-usability, encapsulation) and how to create them (short vs. powerful syntax).

We also took advantage of the discussion to showcase a few advanced exception handling syntax. Finally, we discussed the Singleton pattern and how to use it together with context manager to get a telemetric report.

**Dunderscore**:
- `__enter__`
- `__exit__`
- `__new__`